# CHOCO CRUNCHY ANALYSIS

In [ ]:
import pandas as pd
import requests

all_records = []
page = 1
while page<=120:
    params = {
        "size": 100,
        "page": page
    }
    response = requests.get(f'https://world.openfoodfacts.org/api/v2/search?categories=chocolates&fields=code,product_name,brands,nutriments&page_size=100&page={page}', params= params).json()
    records = response.get('products', [])
    all_records.extend(records)
    page += 1
df = pd.DataFrame(all_records)

In [ ]:
df.info()

In [ ]:
df.head()

In [ ]:
nutriments_df = pd.json_normalize(df['nutriments'])

In [ ]:
nutriments_df.columns

In [ ]:
req_nutriments_df = nutriments_df[['energy-kcal', 'energy-kj','carbohydrates','sugars','fat','saturated-fat','proteins','fiber','salt','sodium','nova-group','nutrition-score-fr','fruits-vegetables-nuts-estimate-from-ingredients']]

In [ ]:
req_nutriments_df.info()

In [ ]:
tot_df = pd.concat([df[['code','product_name','brands']], req_nutriments_df], axis=1)

In [ ]:
tot_df.info()

In [ ]:
tot_df.head()

In [ ]:
# 1. Sugar-to-Carb Ratio
tot_df["sugar_to_carb_ratio"] = tot_df["sugars"] / tot_df["carbohydrates"]
tot_df["sugar_to_carb_ratio"] = tot_df["sugar_to_carb_ratio"].replace([float("inf"), -float("inf")], None)

# 2. Calorie Category
def calorie_category(kcal):
    if pd.isna(kcal):
        return None
    elif kcal <= 100:
        return "Low"
    elif kcal <= 200:
        return "Moderate"
    else:
        return "High"

tot_df["calorie_category"] = tot_df["energy-kcal"].apply(calorie_category)

# 3. Sugar Category
def sugar_category(sugar):
    if pd.isna(sugar):
        return None
    elif sugar <= 5:
        return "Low Sugar"
    elif sugar <= 15:
        return "Moderate Sugar"
    else:
        return "High Sugar"

tot_df["sugar_category"] = tot_df["sugars"].apply(sugar_category)

# 4. Ultra-Processed Flag
tot_df["is_ultra_processed"] = tot_df["nova-group"].apply(
    lambda x: "Yes" if x == 4 else ("No" if pd.notna(x) else None)
)


In [ ]:
tot_df.info()

EDA

In [ ]:
tot_df.shape

In [ ]:
tot_df.describe()

In [ ]:
print(f"Total number od Brands: {tot_df['brands'].nunique()}")
print(f"Total number od Product: {tot_df['product_name'].nunique()}")

Missing Values

In [ ]:
# Count missing values per column
missing = tot_df.isnull().sum().sort_values(ascending=False)
print("Missing values:\n", missing)

# Percentage of missing values
missing_percent = (tot_df.isnull().mean() * 100).round(2)
print("Missing %:\n", missing_percent)


Distributions of Key Nutrients

In [ ]:
import matplotlib.pyplot as plt

# Histogram of energy (kcal)
tot_df["energy-kcal"].dropna().hist(bins=30, edgecolor="black")
plt.title("Distribution of Energy (kcal)")
plt.xlabel("Calories (kcal per 100g)")
plt.ylabel("Count")
plt.show()

# Histogram of sugars
tot_df["sugars"].dropna().hist(bins=30, edgecolor="black")
plt.title("Distribution of Sugars (g/100g)")
plt.xlabel("Sugars (g)")
plt.ylabel("Count")
plt.show()

# Histogram of sugar-to-carb ratio
tot_df["sugar_to_carb_ratio"].dropna().hist(bins=30, edgecolor="black")
plt.title("Distribution of Sugar-to-Carb Ratio")
plt.xlabel("Ratio")
plt.ylabel("Count")
plt.show()


Category-Based Analysis

In [ ]:
# Bar plot for calorie categories
tot_df["calorie_category"].value_counts().plot(kind="bar", color="skyblue", edgecolor="black")
plt.title("Number of Products by Calorie Category")
plt.ylabel("Count")
plt.show()

# Bar plot for sugar categories
tot_df["sugar_category"].value_counts().plot(kind="bar", color="lightcoral", edgecolor="black")
plt.title("Number of Products by Sugar Category")
plt.ylabel("Count")
plt.show()

# Pie chart for NOVA groups
tot_df["nova-group"].value_counts().plot(kind="pie", autopct='%1.1f%%', startangle=90, colors=["lightgreen","orange","skyblue","red"])
plt.title("Proportion of Products by NOVA Group")
plt.ylabel("")
plt.show()

# Compare ultra-processed vs others
tot_df["is_ultra_processed"].value_counts().plot(kind="bar", color="purple", edgecolor="black")
plt.title("Ultra-Processed vs Others")
plt.ylabel("Count")
plt.show()


Relationships

In [ ]:
import seaborn as sns

# Scatter: Calories vs Sugars
sns.scatterplot(x="energy-kcal", y="sugars", data=tot_df, hue="is_ultra_processed", alpha=0.7)
plt.title("Calories vs Sugars (colored by processing level)")
plt.show()

# Boxplot: Calories by Brand (top 10 brands)
top_brands = tot_df["brands"].value_counts().head(10).index
sns.boxplot(x="brands", y="energy-kcal", data=tot_df[tot_df["brands"].isin(top_brands)])
plt.xticks(rotation=45)
plt.title("Calories per 100g across Top 10 Brands")
plt.show()

# Correlation Heatmap
plt.figure(figsize=(10,6))
sns.heatmap(tot_df[["energy-kcal", "sugars", "carbohydrates", "fat", "proteins", "fiber", "salt", "sugar_to_carb_ratio"]].corr(), annot=True, cmap="coolwarm", center=0)
plt.title("Correlation Heatmap of Nutritional Values")
plt.show()


Brand-Level insights

In [ ]:
# Top 10 brands with highest average calories
brand_calories = tot_df.groupby("brands")["energy-kcal"].mean().sort_values(ascending=False).head(10)
brand_calories.plot(kind="bar", color="brown", edgecolor="black")
plt.title("Top 10 Brands by Average Calories")
plt.ylabel("Avg kcal per 100g")
plt.show()

# Top 10 brands with highest average sugar
brand_sugar = tot_df.groupby("brands")["sugars"].mean().sort_values(ascending=False).head(10)
brand_sugar.plot(kind="bar", color="darkred", edgecolor="black")
plt.title("Top 10 Brands by Average Sugar Content")
plt.ylabel("Avg sugars (g per 100g)")
plt.show()


SQL Database

In [ ]:
product_df = tot_df[['code','product_name','brands']]
nutrient_df = tot_df[['code','energy-kcal', 'energy-kj','carbohydrates','sugars','fat','saturated-fat','proteins','fiber','salt','sodium','nova-group','nutrition-score-fr','fruits-vegetables-nuts-estimate-from-ingredients']]
derived_df = tot_df[['code','sugar_to_carb_ratio','calorie_category','sugar_category','is_ultra_processed']]


In [ ]:
import sqlite3

# ----------------- Connect / Create DB -----------------
conn = sqlite3.connect("chocolates.db")  # will create if not exists
cursor = conn.cursor()

# ----------------- Create Tables -----------------
cursor.execute('''CREATE TABLE IF NOT EXISTS product_info (
                    product_code TEXT PRIMARY KEY,
                    product_name TEXT,
                    brand TEXT)''')

cursor.execute('''CREATE TABLE IF NOT EXISTS nutrient_info (
                    product_code TEXT,
                    energy_kcal_value FLOAT,
                    energy_kj_value FLOAT,
                    carbohydrates_value FLOAT,
                    sugars_value FLOAT,
                    fat_value FLOAT,
                    saturated_fat_value FLOAT,
                    proteins_value FLOAT,
                    fiber_value FLOAT,
                    salt_value FLOAT,
                    sodium_value FLOAT,
                    fruits_vegetables_nuts_estimate_100g FLOAT,
                    nutrition_score_fr INTEGER,
                    nova_group INTEGER,
                    FOREIGN KEY (product_code) REFERENCES product_info(product_code))''')

cursor.execute('''CREATE TABLE IF NOT EXISTS derived_metrics (
                    product_code TEXT,
                    sugar_to_carb_ratio FLOAT,
                    calorie_category TEXT,
                    sugar_category TEXT,
                    is_ultra_processed TEXT,
                    FOREIGN KEY (product_code) REFERENCES product_info(product_code))''')

conn.commit()

# ----------------- Insert Data -----------------
# Example DataFrames: product_df, nutrient_df, derived_df
for _, row in product_df.iterrows():
    cursor.execute('''INSERT OR REPLACE INTO product_info (product_code, product_name, brand)
                      VALUES (?, ?, ?)''',
                   (row['code'], row['product_name'], row['brands']))

for _, row in nutrient_df.iterrows():
    cursor.execute('''INSERT OR REPLACE INTO nutrient_info
                      (product_code, energy_kcal_value, energy_kj_value, carbohydrates_value,
                       sugars_value, fat_value, saturated_fat_value, proteins_value, fiber_value,
                       salt_value, sodium_value, fruits_vegetables_nuts_estimate_100g,
                       nutrition_score_fr, nova_group)
                      VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)''',
                   (row['code'], row['energy-kcal'], row['energy-kj'], row['carbohydrates'],
                    row['sugars'], row['fat'], row['saturated-fat'], row['proteins'], row['fiber'],
                    row['salt'], row['sodium'], row['fruits-vegetables-nuts-estimate-from-ingredients'],
                    row['nutrition-score-fr'], row['nova-group']))

for _, row in derived_df.iterrows():
    cursor.execute('''INSERT OR REPLACE INTO derived_metrics
                      (product_code, sugar_to_carb_ratio, calorie_category, sugar_category, is_ultra_processed)
                      VALUES (?, ?, ?, ?, ?)''',
                   (row['code'], row['sugar_to_carb_ratio'], row['calorie_category'],
                    row['sugar_category'], row['is_ultra_processed']))

conn.commit()
#conn.close()


In [ ]:
def execute_query(query):
    query_df = pd.read_sql(query, conn)
    return query_df

product_info Table Queries
1. Count products per brand

In [ ]:
query_1 = """SELECT brand, COUNT(*) AS product_count
FROM product_info
GROUP BY brand;"""
print(execute_query(query_1))

2. Count unique products per brand

In [ ]:
query_2  = """SELECT brand, COUNT(DISTINCT product_name) AS unique_products
FROM product_info
GROUP BY brand;"""
print(execute_query(query_2))

3. Top 5 brands by product count

In [ ]:
query_3 = """SELECT brand, COUNT(*) AS product_count
FROM product_info
GROUP BY brand
ORDER BY product_count DESC
LIMIT 5;
"""
print(execute_query(query_3))

4. Products with missing product name

In [ ]:
query_4 ="""SELECT *
FROM product_info
WHERE product_name IS NULL OR product_name = '';
"""
print(execute_query(query_4))

5. Number of unique brands

In [ ]:
query_5 = """SELECT COUNT(DISTINCT brand) AS unique_brands
FROM product_info;
"""
print(execute_query(query_5))

6. Products with code starting with '3'

In [ ]:
query_6 = """SELECT *
FROM product_info
WHERE product_code LIKE '3%';
"""
print(execute_query(query_6))

nutrient_info Table Queries

1. Top 10 products with highest energy-kcal_value

In [ ]:
query_1 = """SELECT product_code, energy_kcal_value
FROM nutrient_info
ORDER BY energy_kcal_value DESC
LIMIT 10;
"""
print(execute_query(query_1))

2. Average sugars_value per nova-group

In [ ]:
query_2 = """SELECT nova_group, AVG(sugars_value) AS avg_sugars
FROM nutrient_info
GROUP BY nova_group;
"""
print(execute_query(query_2))

3. Count products with fat_value > 20g

In [ ]:
query_3 = """SELECT COUNT(*) AS high_fat_products
FROM nutrient_info
WHERE fat_value > 20;
"""
print(execute_query(query_3))

4. Average carbohydrates_value per product

In [ ]:
query_4 = """SELECT AVG(carbohydrates_value) AS avg_carbs
FROM nutrient_info;
"""
print(execute_query(query_4))

5. Products with sodium_value > 1g

In [ ]:
query_5 ="""SELECT *
FROM nutrient_info
WHERE sodium_value > 1;
"""
print(execute_query(query_5))

6. Count products with non-zero fruits/vegetables/nuts content

In [ ]:
query_6 ="""SELECT COUNT(*) AS products_with_fvn
FROM nutrient_info
WHERE fruits_vegetables_nuts_estimate_100g > 0;
"""
print(execute_query(query_6))

7. Products with energy-kcal_value > 500

In [ ]:
query_7 ="""SELECT *
FROM nutrient_info
WHERE energy_kcal_value > 500;
"""
print(execute_query(query_7))

derived_metrics Table Queries

1. Count products per calorie_category

In [ ]:
query_1 = """SELECT calorie_category, COUNT(*) AS product_count
FROM derived_metrics
GROUP BY calorie_category;
"""
print(execute_query(query_1))

2. Count of High Sugar products

In [ ]:
query_2 = """SELECT COUNT(*) AS high_sugar_products
FROM derived_metrics
WHERE sugar_category = 'High Sugar';
"""
print(execute_query(query_2))

3. Average sugar_to_carb_ratio for High Calorie products

In [ ]:
query_3  = """SELECT AVG(sugar_to_carb_ratio) AS avg_ratio
FROM derived_metrics
WHERE calorie_category = 'High';
"""
print(execute_query(query_3))

4. Products that are both High Calorie and High Sugar

In [ ]:
query_4 =  """SELECT *
FROM derived_metrics
WHERE calorie_category = 'High' AND sugar_category = 'High Sugar';
"""
print(execute_query(query_4))


5. Number of products marked as ultra-processed

In [ ]:
query_5 = """SELECT COUNT(*) AS ultra_processed_count
FROM derived_metrics
WHERE is_ultra_processed = 'Yes';
"""
print(execute_query(query_5))

6. Products with sugar_to_carb_ratio > 0.7

In [ ]:
query_6 ="""SELECT *
FROM derived_metrics
WHERE sugar_to_carb_ratio > 0.7;
"""
print(execute_query(query_6))

7. Average sugar_to_carb_ratio per calorie_category

In [ ]:
query_7 = """SELECT calorie_category, AVG(sugar_to_carb_ratio) AS avg_ratio
FROM derived_metrics
GROUP BY calorie_category;
"""
print(execute_query(query_7))

Join Queries (product_info + nutrient_info + derived_metrics)

1. Top 5 brands with most High Calorie products

In [ ]:
query_1 = """SELECT p.brand, COUNT(*) AS high_calorie_count
FROM product_info p
JOIN derived_metrics d ON p.product_code = d.product_code
WHERE d.calorie_category = 'High'
GROUP BY p.brand
ORDER BY high_calorie_count DESC
LIMIT 5;
"""
print(execute_query(query_1))

2. Average energy-kcal_value for each calorie_category

In [ ]:
query_2 = """SELECT d.calorie_category, AVG(n.energy_kcal_value) AS avg_energy
FROM derived_metrics d
JOIN nutrient_info n ON d.product_code = n.product_code
GROUP BY d.calorie_category;
"""
print(execute_query(query_2))

3. Count of ultra-processed products per brand

In [ ]:
query_3 = """SELECT p.brand, COUNT(*) AS ultra_processed_count
FROM product_info p
JOIN derived_metrics d ON p.product_code = d.product_code
WHERE d.is_ultra_processed = 'Yes'
GROUP BY p.brand;
"""
print(execute_query(query_3))

4. Products with High Sugar and High Calorie along with brand

In [ ]:
query_4 ="""SELECT p.brand, p.product_name, d.calorie_category, d.sugar_category
FROM product_info p
JOIN derived_metrics d ON p.product_code = d.product_code
WHERE d.calorie_category = 'High' AND d.sugar_category = 'High Sugar';
"""
print(execute_query(query_4))

5. Average sugar content per brand for ultra-processed products

In [ ]:
query_5 = """SELECT p.brand, AVG(n.sugars_value) AS avg_sugar
FROM product_info p
JOIN nutrient_info n ON p.product_code = n.product_code
JOIN derived_metrics d ON p.product_code = d.product_code
WHERE d.is_ultra_processed = 'Yes'
GROUP BY p.brand;
"""
print(execute_query(query_5))

6. Number of products with fruits/vegetables/nuts content in each calorie_category

In [ ]:
query_6 = """SELECT d.calorie_category, COUNT(*) AS fvn_products
FROM derived_metrics d
JOIN nutrient_info n ON d.product_code = n.product_code
WHERE n.fruits_vegetables_nuts_estimate_100g > 0
GROUP BY d.calorie_category;
"""
print(execute_query(query_6))

7. Top 5 products by sugar_to_carb_ratio with their calorie and sugar category

In [ ]:
query_7 = """SELECT p.product_name, d.sugar_to_carb_ratio, d.calorie_category, d.sugar_category
FROM product_info p
JOIN derived_metrics d ON p.product_code = d.product_code
ORDER BY d.sugar_to_carb_ratio DESC
LIMIT 5;
"""
print(execute_query(query_7))